## **Data Preparation**

In [163]:
import pandas as pd 

df=pd.read_excel('https://data.go.th/dataset/5370f16a-b553-45ca-929d-7db9a6fc5043/resource/48039a2a-2f01-448c-b2a2-bb0d541dedcd/download/tambon.xlsx')
tambon_df=df.groupby('TA_ID').mean().join(df[['TA_ID','TAMBON_E']].set_index('TA_ID'))[['LAT','LONG','TAMBON_E']]
tambon_df.drop_duplicates(subset=['LAT','LONG'],inplace=True)
tambon_df.to_csv('tambon_location.csv')

/var/folders/zn/20g6j7t91810cl_ts2h592y40000gn/T/ipykernel_53361/2486149103.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tambon_df=df.groupby('TA_ID').mean().join(df[['TA_ID','TAMBON_E']].set_index('TA_ID'))[['LAT','LONG','TAMBON_E']]


### Choose capacity and number of tambons

In [174]:
capacity = 1000
n = 100

In [175]:
import random 

random.seed(31)

tambons=list(tambon_df.index)
random.shuffle(tambons)
tambons=tambons[:n]
tambons=[[tambon,(int(random.random()*6+5)*10)] for tambon in tambons]
tambons[0][1]=0

In [176]:
tambons

[[330207, 0],
 [350306, 100],
 [810115, 90],
 [220906, 80],
 [140615, 60],
 [800502, 80],
 [600510, 60],
 [260310, 100],
 [670117, 70],
 [700110, 80],
 [310804, 80],
 [380204, 90],
 [250202, 90],
 [470301, 50],
 [810406, 70],
 [550502, 70],
 [551502, 90],
 [710503, 70],
 [140706, 70],
 [730602, 100],
 [230202, 70],
 [730104, 70],
 [841505, 70],
 [110104, 70],
 [341112, 90],
 [860112, 70],
 [700302, 50],
 [170107, 100],
 [451606, 100],
 [670701, 70],
 [340717, 70],
 [580408, 60],
 [342504, 80],
 [800204, 80],
 [471401, 60],
 [340104, 90],
 [810804, 80],
 [330910, 60],
 [450703, 70],
 [331501, 70],
 [600604, 100],
 [140101, 60],
 [650605, 100],
 [560301, 80],
 [940310, 90],
 [950302, 80],
 [480401, 70],
 [620607, 90],
 [900701, 60],
 [340406, 100],
 [380103, 80],
 [620602, 100],
 [320507, 50],
 [960201, 100],
 [720509, 100],
 [150105, 60],
 [550511, 50],
 [960901, 50],
 [700701, 80],
 [730615, 50],
 [460205, 50],
 [620117, 70],
 [810505, 60],
 [160614, 100],
 [200620, 50],
 [370606, 50],

## **Your Tasks**

Write a program which can create a route that a truck can deliver items to all places stored in `tambons` list. The number of items that the truck must deliver is the second element in the list. 

The truck has capacity as specified in the variable `capacity`. When it deliver all items, it must go back to the first location in the list to load the items and start deliver again. The process will repeatly be done until all items are delivered. 

For example, if the data in tambons are: 

```
[[400109, 0],
 [730114, 90],
 [101602, 100],
 [320701, 100],
 [190112, 60],
 [330203, 50],
 [571701, 100]]
 ```

 The starting point is tambon `400109` and if the capacity is `300`. One route can be `[400109,101602,320701,571701,400109,730114,190112,330203,400109]` which means 

 1. load `300 (100+100+100)` items and deliver to `101602,320701,571701` respectively
 2. go back to starting point `400109`
 3. load `200 (90+60+50)` items and deliver to `730114,190112,330203`
 4. go back to starting point 



 ### **Scoring scriteria** (from 50 marks)

 1. Use only techniques and codes from the class (0 marks for codes from other resources)
 2. Can find the route to all locations and total distance (in km.) (35 marks)
 3. The route is shortest (40 marks) 
 4. Can find the route which follows the capacity criteria (not neccessary to be the shortest (45 marks)
 5. Can find the shortest route which follow the capacity criteria (50 marks)

### Find km from lat and long 

```
from geopy.distance import geodesic

lat1, lon1 = 40.6892, -74.0445
lat2, lon2 = 37.7749, -122.4194
distance = geodesic((lat1, lon1), (lat2, lon2)).km
```

### Get lat and long from tambon_df 

```
tambon_df.loc[tambon_id]
```

For example,
```
print(tambon_df.loc[571701])
```

# Main

In [177]:
from geopy.distance import geodesic

In [178]:
class DeliveryRoute:
    def __init__(self,n, capacity, tambons):
        self.n = n
        self.capacity = capacity
        self.tambons = tambons
        self.total_distance = 0
        self.current_capacity = capacity
        self.start_tambon = tambons[0][0]
        self.visited_tambons = [tambons[0][0]]
        self.unvisited_tambons = [tambon for tambon in tambons[1:]]

    def get_distance(self, tambon1, tambon2):
        lat1, long1 = tambon_df.loc[tambon1][['LAT','LONG']]
        lat2, long2 = tambon_df.loc[tambon2][['LAT','LONG']]
        # convert to float
        lat1 = float(lat1)
        long1 = float(long1)
        lat2 = float(lat2)
        long2 = float(long2)
        return geodesic((float(lat1), float(long1)), (float(lat2), float(long2))).km
    
    def get_next_tambon_idx(self):
        for idx in range(len(self.unvisited_tambons)):
            load = self.unvisited_tambons[idx][1]
            if self.current_capacity >= load:
                return idx
        return -1
    
    def find_route(self):
        while True:
            if len(self.visited_tambons) == 0:
                print('No solution')
                break
            if len(self.unvisited_tambons) == 0:
                print('Route found!!!')
                distance = self.get_distance(self.visited_tambons[-1], self.start_tambon)
                self.total_distance += distance
                self.visited_tambons.append(self.start_tambon)
                print('Total distance: ', self.total_distance)
                print('Visited tambons: ', self.visited_tambons)
                break
            # Sort unvisited tambons by its load from high to low
            self.unvisited_tambons = sorted(self.unvisited_tambons, key=lambda x: x[1], reverse=True)
            next_tambon_idx = self.get_next_tambon_idx()
            if next_tambon_idx == -1:
                distance = self.get_distance(self.visited_tambons[-1], self.start_tambon)
                self.total_distance += distance
                self.current_capacity = self.capacity
                self.visited_tambons.append(self.start_tambon)
            else:
                self.current_capacity -= self.unvisited_tambons[next_tambon_idx][1]
                distance = self.get_distance(self.visited_tambons[-1], self.unvisited_tambons[next_tambon_idx][0])
                self.total_distance += distance
                self.visited_tambons.append(self.unvisited_tambons.pop(next_tambon_idx)[0])
                
                

In [179]:
# This time find route with the shortest distance
class ShortestDeliveryRoute(DeliveryRoute):
    def get_next_tambon_idx(self):
        min_distance = 100000000
        min_idx = -1
        for idx in range(len(self.unvisited_tambons)):
            load = self.unvisited_tambons[idx][1]
            if self.current_capacity >= load:
                distance = self.get_distance(self.visited_tambons[-1], self.unvisited_tambons[idx][0])
                if distance < min_distance:
                    min_distance = distance
                    min_idx = idx
        return min_idx
    def find_route(self):
        while True:
            if len(self.visited_tambons) == 0:
                print('No solution')
                break
            if len(self.unvisited_tambons) == 0:
                print('Route found!!!')
                distance = self.get_distance(self.visited_tambons[-1], self.start_tambon)
                self.total_distance += distance
                self.visited_tambons.append(self.start_tambon)
                print('Total distance: ', self.total_distance)
                print('Visited tambons: ', self.visited_tambons)
                break
            # Sort unvisited tambons by its load from high to low
            self.unvisited_tambons = sorted(self.unvisited_tambons, key=lambda x: x[1], reverse=True)
            next_tambon_idx = self.get_next_tambon_idx()
            if next_tambon_idx == -1:
                distance = self.get_distance(self.visited_tambons[-1], self.start_tambon)
                self.total_distance += distance
                self.current_capacity = self.capacity
                self.visited_tambons.append(self.start_tambon)
            else:
                self.current_capacity -= self.unvisited_tambons[next_tambon_idx][1]
                distance = self.get_distance(self.visited_tambons[-1], self.unvisited_tambons[next_tambon_idx][0])
                self.total_distance += distance
                self.visited_tambons.append(self.unvisited_tambons.pop(next_tambon_idx)[0])
            
    

In [180]:
deliverRoute = DeliveryRoute(n, capacity, tambons)
deliverRoute.find_route()

Route found!!!
Total distance:  56196.89009382384
Visited tambons:  [330207, 350306, 260310, 730602, 170107, 451606, 600604, 650605, 340406, 620602, 960201, 330207, 720509, 160614, 310207, 420204, 560112, 190208, 601108, 410117, 412104, 810115, 330207, 380204, 250202, 551502, 341112, 340104, 940310, 620607, 801216, 820801, 332001, 130108, 330207, 220906, 800502, 700110, 310804, 342504, 800204, 810804, 560301, 950302, 380103, 700701, 660603, 330207, 720505, 670117, 810406, 550502, 710503, 140706, 230202, 730104, 841505, 110104, 860112, 670701, 340717, 450703, 330207, 331501, 480401, 620117, 640711, 390405, 340911, 331302, 240703, 420103, 140615, 600510, 580408, 471401, 330910, 140101, 330207, 900701, 150105, 810505, 610103, 841301, 400711, 901508, 103001, 411810, 320715, 301511, 570405, 920202, 620101, 470301, 700302, 320507, 330207, 550511, 960901, 730615, 460205, 200620, 370606, 160318, 250704, 770404, 380306, 330207]


[400109,101602,320701,571701,400109,730114,190112,330203,400109] 

In [181]:
ShortestDeliveryRoute(n, capacity, tambons).find_route()

Route found!!!
Total distance:  14771.032173123831
Visited tambons:  [330207, 340406, 340104, 341112, 342504, 340717, 331302, 331501, 332001, 330910, 320715, 370606, 350306, 450703, 330207, 340911, 320507, 310804, 310207, 301511, 250202, 250704, 240703, 200620, 110104, 103001, 130108, 140615, 140101, 330207, 451606, 460205, 470301, 471401, 480401, 380204, 380306, 380103, 410117, 412104, 400711, 390405, 420204, 330207, 670701, 670117, 660603, 600510, 600604, 601108, 610103, 160614, 160318, 170107, 720505, 720509, 330207, 411810, 420103, 650605, 640711, 620602, 620607, 620101, 620117, 150105, 140706, 190208, 260310, 730615, 330207, 220906, 230202, 730602, 730104, 700701, 700110, 710503, 700302, 770404, 860112, 841301, 841505, 800502, 810406, 330207, 550511, 551502, 550502, 560301, 570405, 560112, 580408, 820801, 810505, 810115, 810804, 920202, 900701, 960901, 330207, 800204, 801216, 901508, 940310, 950302, 960201, 330207]
